In [ ]:
import numpy as np
import pandas as pd 
%matplotlib inline

In [ ]:
!pip install dgl

In [ ]:
#adding features and formatting data
import scipy.sparse as sp

def load_data(path="../input/terrorattack/TerrorAttack/", dataset="TerrorAttack"):
    """Load citation network dataset (cora only for now)"""
    print('Loading {} dataset...'.format(dataset))
    
    idx_features_labels = np.genfromtxt("../input/terrorattack/TerrorAttack/terrorist_attack.nodes",
                                        np.dtype(str),comments=None)

    print(idx_features_labels)
    
    features = sp.csr_matrix(idx_features_labels[:, 1:-1], dtype=np.float32)
    print(features)
    
    labels = encode_onehot(idx_features_labels[:, -1])
    
    idx = np.array(idx_features_labels[:, 0], dtype=None)

    idx_map = {j: i for i, j in enumerate(idx)}
    
    edges_unordered = np.genfromtxt("../input/terrorattack/TerrorAttack/terrorist_attack_loc.edges",
                                    np.dtype(str),comments=None)
    
    edges = np.array(list(map(idx_map.get, edges_unordered.flatten())),
                     dtype=np.int32).reshape(edges_unordered.shape)
    
    adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                        shape=(labels.shape[0], labels.shape[0]),
                        dtype=np.float32)
    
    adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

    features = normalize(features)
    adj = normalize(adj + sp.eye(adj.shape[0]))
    
    idx_train = range(1200) #originally (180)
    idx_val = range(500, 1000) #originally (200,500)
    idx_test = range(1000, 1293) #originally (500,1500)

    features = torch.FloatTensor(np.array(features.todense()))
    labels = torch.LongTensor(np.where(labels)[1])
    adj = sparse_mx_to_torch_sparse_tensor(adj)

    idx_train = torch.LongTensor(idx_train)
    idx_val = torch.LongTensor(idx_val)
    idx_test = torch.LongTensor(idx_test)
    
    return adj, features, labels, idx_train, idx_val, idx_test

def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

all_lab=[]
def accuracy(output, labels):
    all_lab.append(labels)
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)


def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

In [ ]:
#one hot encoding
import torch.nn as nn
import torch.nn.functional as F

def encode_onehot(labels):
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in
                    enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)),
                         dtype=np.int32)
    print(labels_onehot)
    return labels_onehot

In [ ]:
#graph format
import dgl
import scipy.sparse as sp
import networkx as nx
g = dgl.DGLGraph()

g.add_nodes(1293)

idx_features_labels = np.genfromtxt("../input/terrorattack/TerrorAttack/terrorist_attack.nodes",

                                        np.dtype(str),comments=None)


features = sp.csr_matrix(idx_features_labels[:, 1:-1], dtype=np.float32)

labels = encode_onehot(idx_features_labels[:, -1])
idx = np.array(idx_features_labels[:, 0], dtype=None)

idx_map = {j: i for i, j in enumerate(idx)}

edges_unordered = np.genfromtxt("../input/terrorattack/TerrorAttack/terrorist_attack_loc.edges",

                                np.dtype(str),comments=None);
edges = np.array(list(map(idx_map.get, edges_unordered.flatten())),dtype=np.int32).reshape(edges_unordered.shape)

src, dst = tuple(zip(*edges));

g.add_edges(src, dst);

g.add_edges(dst, src);

print('We have %d nodes.' % g.number_of_nodes())

print('We have %d edges.' % g.number_of_edges())

nx_G = g.to_networkx().to_undirected()
pos = nx.kamada_kawai_layout(nx_G)

nx.draw(nx_G, pos, with_labels=True, node_color=[[.7, .7, .7]])

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
# Define the message & reduce function
# NOTE: we ignore the GCN's normalization constant c_ij for this tutorial.
def gcn_message(edges):
    # The argument is a batch of edges.
    # This computes a (batch of) message called 'msg' using the source node's feature 'h'.
    return {'msg' : edges.src['h']}

def gcn_reduce(nodes):
    # The argument is a batch of nodes.
    # This computes the new 'h' features by summing received 'msg' in each node's mailbox.
    return {'h' : torch.sum(nodes.mailbox['msg'], dim=1)}

# Define the GCNLayer module
class GCNLayer(nn.Module):
    def __init__(self, in_feats, out_feats):
        super(GCNLayer, self).__init__()
        self.linear = nn.Linear(in_feats, out_feats)

    def forward(self, g, inputs):
        # g is the graph and the inputs is the input node features
        # first set the node features
        g.ndata['h'] = inputs
        # trigger message passing on all edges 
        g.send(g.edges(), gcn_message)
        # trigger aggregation at all nodes
        g.recv(g.nodes(), gcn_reduce)
        # get the result node features
        h = g.ndata.pop('h')
        # perform linear transformation
        return self.linear(h)

In [ ]:
# Define a 2-layer GCN model
class GCN(nn.Module):
    def __init__(self, in_feats, hidden_size, num_classes):
        super(GCN, self).__init__()
        self.gcn1 = GCNLayer(in_feats, hidden_size)
        self.gcn2 = GCNLayer(hidden_size, num_classes)

    def forward(self, g, inputs):
        h = self.gcn1(g, inputs)   
        h = torch.relu(h)
        h = self.gcn2(g, h)
        return h

net = GCN(1293, 16, 6)
print(net)

In [ ]:
import torch
inputs = torch.eye(1293)
print(inputs)

labeled_nodes = torch.tensor([200,400, 600,800,1000,0])  
print(labeled_nodes)
labels = torch.tensor([0, 1,2,3,4,5])
print(labels)# their labels are different

In [ ]:
#training
optimizer = torch.optim.Adam(net.parameters(), lr=0.01)
all_logits = []
for epoch in range(205):
    logits = net(g, inputs)
    all_logits.append(logits.detach())
  
    logp = F.log_softmax(logits, 1)

    loss = F.nll_loss(logp[labeled_nodes], labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print('Epoch %d | Loss: %.4f' % (epoch, loss.item()))

In [ ]:
#reducing dimentionality
from sklearn.manifold import TSNE
from matplotlib import pyplot as plt
tsne_data1=[]
i=0
epochess=[]
while(i<201):
    outp=all_logits[i]
    data_1000=outp
    md=TSNE(n_components=2,random_state=0,perplexity = 50,learning_rate  = 100, n_iter=5000)
    tsne_data=md.fit_transform(data_1000)
    tsne_data1.append(tsne_data.tolist())
    epochess.append(i)
    i=i+1

In [ ]:
print(len(tsne_data1[0][0]))

In [ ]:
import matplotlib.animation as animation
import matplotlib.pyplot as plt
epoc=[]
group=[]
def draw(i):
    cls1color = '#9B59B6'
    cls2color = '#F7DC6F'
    cls3color = 'black'
    cls4color = 'yellow'
    cls5color = 'green'
    cls6color = 'red'
    pos = {}
    dos = {}
    colors = []
    lab=[]
    for v in range(1293):
        ld=[]
        dos[v] = all_logits[i][v].numpy()
        cls = dos[v].argmax() # returns index of highest 
        lab.append(cls)
        a=np.array(tsne_data1[i][v])
        pos[v] = tsne_data1[i][v]
        if(cls==0):
            colors.append(cls1color)
        elif(cls==1):
            colors.append(cls2color)
        elif(cls==2):
            colors.append(cls3color)
        elif(cls==3):
            colors.append(cls4color)  
        elif(cls==4):
            colors.append(cls5color)
        elif(cls==5):
            colors.append(cls6color) 
        epoc.append(i)
        #print(pos)
    group.append(lab)
    ax.cla()
    ax.axis('off')
    ax.set_title('Epoch: %d' % i)
    nx.draw_networkx(nx_G.to_undirected(), pos, node_color=colors,
           with_labels=False, node_size=10, ax=ax)
    
fig = plt.figure(dpi=150)
fig.clf()
ax = fig.subplots()

draw(200)
plt.show()

In [ ]:
ani = animation.FuncAnimation(fig, draw, frames=len(all_logits), interval=201)
ani.save('pickup_animation.gif', writer='imagemagick', fps=10)

In [ ]:
#writing data to csv
from sklearn.manifold import TSNE
from matplotlib import pyplot as plt
import pandas as pd
i = 0
while i<201:
    outp=all_logits[i]
    data_1000=outp
    print(group[i])
    md=TSNE(n_components=2,random_state=0,perplexity = 50,learning_rate  = 100, n_iter=5000)
    tsne_data1=md.fit_transform(data_1000)#Apply TSNE on the 190 iter
    tsne_data1=np.vstack((group[i],tsne_data1.T)).T #Stack 
    tsne_df=pd.DataFrame(data=tsne_data1,columns=["label","Dim_1","Dim_2"])
    tsne_df["Epochs"] = i
    export_csv = tsne_df.to_csv ('files.csv',mode="a", index = None, header=False)
    print("iteraction - ", i)
    i = i+10